In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [17]:
!cp "/content/drive/MyDrive/data/test.zip" '.'
!cp "/content/drive/MyDrive/data/model.h5" '.'
!cp "/content/drive/MyDrive/data/model.json" '.'
!cp "/content/drive/MyDrive/data/sampleSubmission.csv" '.'
!cp "/content/drive/MyDrive/data/sound.mp3" '.'

In [3]:
!unzip -q test.zip 
!rm test.zip

In [20]:
print(len(os.listdir('./test')))

300


In [14]:
import os  
import csv
import cv2
import glob
import random
import numpy as np

from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
from keras.models import model_from_json
from moviepy.editor import CompositeAudioClip, VideoFileClip, AudioFileClip

In [28]:
TEST_DIR = './test'
IMG_SIZE = 128

def csv_save():
  counter=1
  for img in tqdm(os.listdir(TEST_DIR)):
    path = os.path.join(TEST_DIR,img)
    img = cv2.imread(path)    
    img = cv2.resize(img, (128,128))
    img = img.reshape(1, 128, 128, 3)
    with open('./sampleSubmission.csv', 'a+', newline='') as csv_file:
        writer = csv.writer(csv_file)
        if loaded_model.predict(img) == 1:
            writer.writerow([counter, "DOG"])
        else:
            writer.writerow([counter, "CAT"])
    counter += 1

In [29]:
json_file = open('./model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("./model.h5")
loaded_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

with open('sampleSubmission.csv', 'w', newline='') as csv_file:
    csv_file.truncate()

In [44]:
csv_save()

100%|██████████| 300/300 [00:17<00:00, 16.89it/s]


In [8]:
os.mkdir('./test_data')
os.mkdir('./test_data/dogs')
os.mkdir('./test_data/cats')

In [31]:
TEST_DIR = './test'
TEST_DATA_DIR = './test_data'
IMG_SIZE = 500

with open('sampleSubmission.csv', newline='') as csv_file:
    rows = list(csv.reader(csv_file))

In [32]:
def create_dir():
  counter=1
  for img in tqdm(os.listdir(TEST_DIR)):
    path = os.path.join(TEST_DIR,img)
    img = cv2.imread(path)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    if rows[counter-1][1] == 'DOG':
        lable = 'dogs'
        #print('./test_data/' + lable + '/' + str(counter) + '.jpg')
        cv2.imwrite(('./test_data/' + lable + '/' + str(counter) + '.jpg'), img)
    else:
        lable = 'cats'
        #print('./test_data/' + lable + '/' + str(counter) + '.jpg')
        cv2.imwrite(('./test_data/' + lable + '/' + str(counter) + '.jpg'), img)
    counter += 1

In [33]:
create_dir()

100%|██████████| 300/300 [00:02<00:00, 105.35it/s]


In [34]:
print(len(os.listdir('./test_data/dogs')))

184


In [35]:
print(len(os.listdir('./test_data/cats')))

116


In [36]:
def create_mp4():
    path = []
    animal = input('DOG or CAT?')
    
    if animal == 'DOG' or animal == 'dog':    
        pathDir = os.listdir(TEST_DATA_DIR + '/dogs')
        filenumber=len(pathDir)
        sample = random.sample(pathDir, 20)
        #print (sample[0])
        for i in sample:
            path.append("./test_data/dogs/" + i)
    if animal == 'CAT' or animal == 'cat':    
        pathDir = os.listdir(TEST_DATA_DIR + '/cats')
        filenumber=len(pathDir)
        sample = random.sample(pathDir, 20)
        for i in sample:
            path.append("./test_data/cats/" + i)
    
    for i in range(20):
        img = cv2.imread(path[i])
        plt.imshow(img[:,:,::-1])
        plt.show()
    result_name = 'output.mp4'

    frame_list = path
    print("frame count: ",len(frame_list))

    fps = 0.3
    shape = cv2.imread(frame_list[0]).shape # delete dimension 3
    size = (shape[1], shape[0])
    print("frame size: ",size)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(result_name, fourcc, fps, size)
    
    for idx, path in enumerate(frame_list):
        frame = cv2.imread(path)
        current_frame = idx+1
        total_frame_count = len(frame_list)
        percentage = int(current_frame*30 / (total_frame_count+1))
        print("\rProcess: [{}{}] {:06d} / {:06d}".format("#"*percentage, "."*(30-1-percentage), current_frame, total_frame_count), end ='')
        out.write(frame)

    out.release()
    print("\n已儲存MP4")

In [45]:
create_mp4()

Output hidden; open in https://colab.research.google.com to view.

In [46]:
def combine_audio(vidname, audname, outname, fps=60): 
    import moviepy.editor as mpe
    my_clip = mpe.VideoFileClip(vidname)
    audio_background = mpe.AudioFileClip(audname)
    final_clip = my_clip.set_audio(audio_background)
    final_clip.write_videofile(outname,fps=fps)
    
combine_audio("./output.mp4", "./sound.mp3", "./output_with_sound.mp4")

[MoviePy] >>>> Building video ./output_with_sound.mp4
[MoviePy] Writing audio in output_with_soundTEMP_MPY_wvf_snd.mp3


100%|██████████| 1334/1334 [00:01<00:00, 673.54it/s]

[MoviePy] Done.
[MoviePy] Writing video ./output_with_sound.mp4



100%|██████████| 4001/4001 [00:24<00:00, 160.66it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_with_sound.mp4 

